In [1]:
import numpy as np

Lx = 100.0
Lz = 100.0
nx = 256
nz = 256

S = 3

In [2]:
k0 = 2*np.pi/Lx
kx_zonal = k0*1
kz_zonal = 0

Omega = 1

Pr_exp = np.linspace(0.02, 7, 28)
Pr_array = 10**(-Pr_exp)

r_exp = np.linspace(0.02, 5, 20)
r_array  = 10**(-r_exp)



In [3]:
import time
start_time = time.time()
print("--- %s seconds ---" % (start_time))

ux_uy = np.zeros((Pr_array.shape[0], r_array.shape[0]), dtype=np.float64)
ux_th = np.zeros((Pr_array.shape[0], r_array.shape[0]), dtype=np.float64)

Pr_count = 0
for Pr in Pr_array:
    r_count = 0
    for r in r_array:
    
        R0 = 1 + r*(1/Pr-1)
        kappa_epsqrd = 2*Omega*(2*Omega-S)
        Nsqrd = -kappa_epsqrd*R0
        kappa = np.sqrt(Nsqrd/Pr) #This is because the box size is measured in units of d = (nu*kappa/Nsqrd)^(1/4), see Eq. (5) in https://arxiv.org/pdf/1905.06962.pdf.
        nu = kappa*Pr
        typical_distance_d = (nu*kappa/Nsqrd)**(1/4)
        typical_tau = typical_distance_d**2/kappa


        p1 = lambda kx, kz : k2(kx,kz)*(kappa+2*nu)
        p2 = lambda kx, kz : nu*k2(kx,kz)**2*(2*kappa+nu) + (kappa_epsqrd + Nsqrd)*kz**2/k2(kx,kz)
        p3 = lambda kx, kz : ( nu**2*kappa*k2(kx,kz)**2 + (kappa_epsqrd*kappa+Nsqrd*nu)*kz**2/k2(kx,kz) ) * k2(kx,kz)
        gamma = lambda kx, kz : np.max(np.real(np.roots([1, p1(kx,kz), p2(kx,kz), p3(kx,kz)])))

        f = lambda kx, kz : np.sqrt(k2(kx,kz))/gamma(kx,kz)*np.sqrt(k2(kx,kz))/gamma_nu(kx,kz)*typical_distance_d/typical_tau
        k2 = lambda kx, kz : kx**2+kz**2
        gamma_nu = lambda kx, kz: gamma(kx, kz) + nu*k2(kx, kz)
        gamma_kappa = lambda kx, kz: gamma(kx, kz) + kappa*k2(kx, kz)
        tau_11Z = lambda kx_p, kz_p, kx_pp, kz_pp : (gamma(kx_p,kz_p) + gamma(kx_pp,kz_pp) - nu*k2(kx_zonal,kz_zonal))**(-1)
        C_Z11_symm_k_kp = lambda kx_p, kz_p, kx_pp, kz_pp : 1.0j*( kx_p**2-kx_pp**2 )/(kz_p * f(kx_p, kz_p) * f(kx_pp, kz_pp))
        C_1Z1_symm_kpp_k = lambda kx_p, kz_p, kx_pp, kz_pp : 1.0j*kz_p*f(kx_pp,kz_pp)/f(kx_p,kz_p)*(k2(kx_pp, kz_pp)/kz_pp**2*1/Nsqrd*(1-2*kx_zonal*kx_pp/k2(kx_pp,kz_pp)) 
                                                              + 1/R0*1/gamma_nu(kx_p,kz_p)*1/gamma_nu(kx_pp,kz_pp) - 1/gamma_kappa(kx_p,kz_p)*1/gamma_kappa(kx_pp,kz_pp))/( 
                                                              k2(kx_pp, kz_pp)/kz_pp**2*1/Nsqrd + 1/R0*(1/gamma_nu(kx_pp, kz_pp))**2 - (1/gamma_kappa(kx_pp, kz_pp))**2 )

        ######################################   
        summand_uxuy = 0
        for kx_ppp_ind in range(-int(nx/2), int(nx/2)):
            for kz_ppp_ind in range(-int(nz/2), int(nz/2)):
                kx_ppp = kx_ppp_ind*k0
                kz_ppp = kz_ppp_ind*k0
                if k2(kx_ppp, kz_ppp)!=0:
                    if gamma(kx_ppp, kz_ppp)>0:
                        summand_uxuy += -(2*Omega-S)/gamma_nu(kx_ppp,kz_ppp)*1/(f(kx_ppp,kz_ppp)**2)

        ######################################             
        summand_uxth = 0
        for kx_ppp_ind in range(-int(nx/2), int(nx/2)):
            for kz_ppp_ind in range(-int(nz/2), int(nz/2)):
                kx_ppp = kx_ppp_ind*k0
                kz_ppp = kz_ppp_ind*k0
                if k2(kx_ppp, kz_ppp)!=0:
                    if gamma(kx_ppp, kz_ppp)>0:
                        summand_uxth += -Nsqrd/gamma_kappa(kx_ppp,kz_ppp)*1/(f(kx_ppp,kz_ppp)**2)

        ######################################               
        CCTau = 0
        for kx_p_ind in range(-int(nx/2), int(nx/2)):
            for kz_p_ind in range(-int(nz/2), int(nz/2)):
                kx_p = kx_p_ind*k0
                kz_p = kz_p_ind*k0
                if k2(kx_p, kz_p)!=0:
                    if gamma(kx_p, kz_p)>0:
                        kx_pp = kx_zonal - kx_p
                        kz_pp = kz_zonal - kz_p
                        CCTau += -tau_11Z(kx_p, kz_p, kx_pp, kz_pp) * C_Z11_symm_k_kp(kx_p, kz_p, kx_pp, kz_pp) * C_1Z1_symm_kpp_k(kx_p, kz_p, kx_pp, kz_pp)

        ux_uy[Pr_count, r_count] =  1/np.abs(np.real(CCTau)) * (nu*k2(kx_zonal,kz_zonal)) * summand_uxuy
        ux_th[Pr_count, r_count] =  1/np.abs(np.real(CCTau)) * (nu*k2(kx_zonal,kz_zonal)) * summand_uxth
        print("Pr:%0.10f, r:%0.10f, uxuy:%0.10f" %(Pr, r, ux_uy[Pr_count, r_count]))
        r_count += 1
    
    Pr_count += 1
    
end_time = time.time()
print("--- %s seconds ---" % (end_time))